In [1]:
import xml.etree.ElementTree as ET
from nltk.stem import PorterStemmer
import re
from collections import Counter, defaultdict
import numpy as np
import math

In [2]:
f = open('/home/faruk/Desktop/Uni/UoE/TTDS/Lab1/englishST.txt', "r", encoding="utf-8-sig")
STwords = [word.rstrip() for word in f.readlines()]
ps = PorterStemmer()

In [3]:
tree = ET.parse('/home/faruk/Desktop/Uni/UoE/TTDS/Lab2/collections/trec.sample.xml')
root = tree.getroot()

## Create inv index

In [4]:
def create_invindex(file):
    
    tree = ET.parse(file)
    root = tree.getroot()

    inv_index = defaultdict(list)
    docs = []

    for idx,child in enumerate(root.findall('.//DOC')):
        headline = child.find('.//HEADLINE').text
        docnum = child.find('.//DOCNO').text
        text = child.find('.//TEXT').text
        combined = headline.strip() + " " + text
        tks = re.findall(r'\d+\.\d+(?:bn|m)?|\w+', combined)
        docs.append(docnum)
        tks = [ps.stem(word) for word in tks if word.lower() not in STwords]
        for pos, word in enumerate(tks):
            if word in inv_index: 
                if docnum in inv_index[word][1]:
                    inv_index[word][1][docnum].append(pos) # If the the word already ocurred in the docnum, append the position of the new occurrence.
                else:
                    inv_index[word][1][docnum] = [pos] # Else, leave position list untouched.

            else:
                inv_index[word].append('') # If word is not in inv_index, create key and set frequency to 1.
                inv_index[word].append({}) # Declare posting list for new word.   
                inv_index[word][1][docnum] = [pos] # For each word, add document ID : [pos] pair.

            inv_index[word][0] = len(inv_index[word][1].keys()) # If the word already exists in the inv index, increase the frequency of word.
    
    return inv_index, docs

In [5]:
inv_index, docs = create_invindex('/home/faruk/Desktop/Uni/UoE/TTDS/Lab2/collections/trec.sample.xml')

In [ ]:
inv_index

## Boolean search parser

In [9]:
def get_word(inv_index,word):
    if word in inv_index.keys():
        return inv_index[word]
    else:
        return []
    
#get_word(inv_index,'incom')[1].keys()

In [ ]:
def proximity_search(inv_index,word1,word2,distance=1):
    #a = inv_index[word1][1]
    a = get_word(inv_index,word1)[1]
    #b = inv_index[word2][1]
    b = get_word(inv_index,word2)[1]
    match = a.keys() & b.keys()
    newdict = {k:(a[k],b[k]) for k in match}
    documents = set()
    for key,val in newdict.items():
        for i in range(len(val)-1):
            for j in range(len(val[i])):
                temp = val[i][j]
                for k in range(len(val[i+1])):
                    temp2 = val[i+1][k]
                    if distance == 1:
                        if temp2 - temp == distance:
                            documents.add(key)
                    else:
                        if abs(temp2 - temp) <= distance:
                            documents.add(key)
                        

    documents = sorted(list(map(int,documents)))
    return documents    

proximity_search(inv_index, 'incom', 'tax', 10)

In [11]:
def query_parser(query,inv_index):
    print(query)
    query = query.lower()
    
    if query.find(" ") == -1 and query.find('#') == -1:
        word = ps.stem(query)
        docs = get_word(inv_index,word)[1].keys()
        return sorted(map(int,docs))
    
    if query.find('"') != -1:
        tks_query = re.findall(r'"(.+?)"',query)[0]
        tks_query = tks_query.split()
        tks_query = [ps.stem(word) for word in tks_query]
        temp1 = tks_query
        result = proximity_search(inv_index,tks_query[0],tks_query[1])
        if query.find(' and ') != -1:
            if query.find('not') != -1:
                if query.count('"') == 4:
                    tks_query = re.findall(r'"(.+?)"',query)[1]
                    tks_query = tks_query.split()
                    tks_query = [ps.stem(word) for word in tks_query]
                    notword = re.findall(r'not\s"(.+?)"',query)[0].split()
                    notword = [ps.stem(word) for word in notword]
                    if notword == temp1:
                        temp = proximity_search(inv_index,tks_query[0],tks_query[1])
                        return sorted(set(temp).difference(set(result)))
                    else:
                        temp = proximity_search(inv_index,tks_query[0],tks_query[1])
                        return sorted(set(result).difference(set(temp)))
                
                notword = re.findall(r'not\s(.+?)(?:$|\s)',query)[0]
                w = ps.stem(re.sub(r'"(.*)"|and|not|\s','',query))
                w = get_word(inv_index, w)[1].keys()
                if notword.find('"') != -1:
                    return sorted(set(map(int,w)).difference(set(result)))
                else:
                    return sorted(set(result).difference(set(map(int,w))))
            
            else:
                if query.count('"') == 4:
                    tks_query = re.findall(r'"(.+?)"',query)[1]
                    tks_query = tks_query.split()
                    tks_query = [ps.stem(word) for word in tks_query]
                    temp = proximity_search(inv_index,tks_query[0],tks_query[1])
                    return sorted(set(map(int,temp)).intersection(set(result)))
                    
                w = ps.stem(re.sub(r'"(.*)"|and|\s','',query))
                w = get_word(inv_index, w)[1].keys()
                return sorted(set(map(int,w)).intersection(set(result)))
        
        elif query.find(' or ') != -1:
            if query.count('"') == 4:
                tks_query = re.findall(r'"(.+?)"',query)[1]
                tks_query = tks_query.split()
                tks_query = [ps.stem(word) for word in tks_query]
                temp = proximity_search(inv_index,tks_query[0],tks_query[1])
                return sorted(set(result).union(temp))
            else:
                w = ps.stem(re.sub(r'"(.*)"|or|\s','',query))
                w = get_word(inv_index, w)[1].keys()
                return sorted(set(result).union(set(map(int,w))))
        
        else:
            return result
        
    if query.find('#') != -1:
        distance = int(re.findall(r'\d+', query)[0])
        tks_query = re.findall(r'\((.+)\)', query)[0]
        tks_query = tks_query.split(',')
        tks_query = [ps.stem(word.strip()) for word in tks_query]
        return proximity_search(inv_index,tks_query[0],tks_query[1],distance=distance)
    
    if query.find(' and ') != -1:
        if query.find('not') != -1:
            notword = re.findall(r'not\s(.+?)(?:$|\s)',query)[0]
            w = re.sub(notword, '', query)
            w = ps.stem(re.sub(r'and|not', '', w).strip())
            notword = ps.stem(notword)
            notword = set(map(int,get_word(inv_index, notword)[1].keys()))
            w = set(map(int,get_word(inv_index, w)[1].keys()))
            return sorted(w.difference(notword))

        else:
            w1 = set(map(int,get_word(inv_index,ps.stem(re.findall(r'^\w+',query)[0]))[1].keys()))
            w2 = set(map(int,get_word(inv_index, ps.stem(re.findall(r'and\s(.+)',query)[0]))[1].keys()))
            return sorted(w1.intersection(w2))
        
    if query.find(' or ') != -1:
        w1 = set(map(int,get_word(inv_index,ps.stem(re.findall(r'^\w+',query)[0]))[1].keys()))
        w2 = set(map(int,get_word(inv_index,ps.stem(re.findall(r'or\s(.+)',query)[0]))[1].keys()))
        return sorted(w1.union(w2))

In [12]:
print(query_parser('Happiness',inv_index))
print('\n')
print(query_parser('Edinburgh AND Scotland',inv_index))
print('\n')
print(query_parser('income and taxes',inv_index))
print('\n')
print(query_parser('"income taxes"',inv_index))
print('\n')
print(query_parser('#20(income, taxes)',inv_index))
print('\n')
print(query_parser('"middle east" AND peace',inv_index))
print('\n')
print(query_parser('"islam religion"',inv_index))
print('\n')
print(query_parser('"Financial times" AND NOT BBC',inv_index))
print('\n')
print(query_parser('#15(dow,stocks)',inv_index))

Happiness
[58, 136, 137, 196, 264, 290, 341, 372, 3329, 3362, 3443, 3474, 3638, 3730, 3773, 3856, 3864, 3913]


Edinburgh AND Scotland
[351]


income and taxes
[16, 24, 39, 65, 92, 141, 163, 172, 282, 314, 326, 361, 3327, 3343, 3387, 3405, 3441, 3449, 3490, 3495, 3519, 3533, 3535, 3562, 3582, 3589, 3590, 3596, 3599, 3606, 3608, 3617, 3699, 3705, 3706, 3707, 3708, 3710, 3734, 3816, 3817, 3818, 3829, 3834, 3835, 3937]


"income taxes"
[65, 92, 282, 361, 3405, 3441, 3449, 3562, 3706, 3708, 3734, 3818]


#20(income, taxes)
[65, 92, 163, 282, 361, 3387, 3405, 3441, 3449, 3490, 3533, 3562, 3589, 3590, 3599, 3606, 3617, 3699, 3705, 3706, 3708, 3734, 3816, 3817, 3818, 3829, 3834, 3835]


"middle east" AND peace
[219, 223, 288, 305, 3549, 3663, 3762, 3766]


"islam religion"
[]


"Financial times" AND NOT BBC
[38, 55, 133, 140, 146, 306, 347, 3580, 3690, 3692, 3740, 3907]


#15(dow,stocks)
[14, 154]


### Read query and write results to file

In [ ]:
def read_queries_boolean(file, inv_index):
    with open(file, 'r') as file, open('results.boolean.txt', 'w') as outfile:
        for i,line in enumerate(file):
            data = line.strip()
            data = re.sub(r'\d+','',data).strip()
            results = query_parser(data,inv_index)
            for result in results:
                outfile.write(str(i+1) + ',' + str(result) + '\n')
                
read_queries_boolean('collections/queries.lab2.txt', inv_index)

## Ranked search

### Create tfidf document index

In [18]:
def ranked_search(query, inv_index, docs):
    print(query)
    queries = re.findall(r'\w+',query)
    queries = [ps.stem(word) for word in queries if ps.stem(word.lower()) not in STwords]

    query_index = {}
    tfidf_index = {}
    for doc in docs:
        tfidf_index[doc] = {} 

    for word in queries:
        query_index[word] = 1 + math.log(queries.count(word),10)
        for doc in docs:
            # Search which documents contain the word and calculate tfidf, if word not in document, then tfidf=0.
                if doc in inv_index[word][1].keys():
                    tfidf_index[doc][word] = (1 + math.log(len(inv_index[word][1][doc]),10)) * (math.log((len(docs)/inv_index[word][0]),10))
                else:
                    tfidf_index[doc][word] = 0       
                    
    scores = []
    for doc in docs:
        score = 0
        for word in queries:
            if tfidf_index[doc][word] != 0:
                score += query_index[word] * (tfidf_index[doc][word])
        if score > 0:
            scores.append((doc,score))
    
    return sorted(scores, key=lambda tup: tup[1], reverse=True)

In [ ]:
print(ranked_search("income tax reduction", inv_index, docs)[:10])
print('\n')
print(ranked_search("stock market in Japan", inv_index, docs)[:10])
print('\n')
print(ranked_search("health industry", inv_index, docs)[:10])
print('\n')
print(ranked_search("the Robotics industries", inv_index, docs)[:10])
print('\n')
print(ranked_search("the peace process in the middle east", inv_index, docs)[:10])
print('\n')
print(ranked_search("information retrieval", inv_index, docs)[:10])
print('\n')
print(ranked_search("Dow Jones industrial average stocks", inv_index, docs)[:10])
print('\n')
print(ranked_search("will be there a reduction in the income taxes?", inv_index, docs)[:10])
print('\n')
print(ranked_search("the gold prices versus the dollar price", inv_index, docs)[:10])
print('\n')
print(ranked_search("FT article on the BBC and BSkyB deal", inv_index, docs)[:10])

### Read query and write results to file

In [ ]:
def queries_ranked(file, tfidf_index, vocab, doc_index):
    res = []
    with open(file, 'r') as file, open('results.ranked.txt', 'w') as outfile:
        for i,line in enumerate(file):
            data = line.strip()
            data = re.sub(r'^\d+\s','',data).strip()
            results = ranked_search(data,tfidf_index, vocab, doc_index)
            for j,result in enumerate(results):
                if j < 150:
                    outfile.write(str(i+1) + ',' + str(result[0]) + ',' + str(round(result[1],4)) + '\n')
                else:
                    continue
                        
queries_ranked('collections/queries.lab3.txt', tfidf,vocab,doc_index)